In [2]:
import numpy as np
import pandas as pd


In [11]:
import subprocess
import json

def generate_sentence(query):
    # Construct the POST data as a dictionary
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                "content": query
            }
        ],
        "stream": False
    }

    # Convert dictionary to JSON format
    json_data = json.dumps(data)

    # Construct the curl command
    curl_command = [
        'curl',
        'http://192.168.1.124:11434/api/chat',
        '-d', json_data,
        '-H', 'Content-Type: application/json'
    ]

    # Execute the curl command
    process = subprocess.Popen(curl_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = process.communicate()

    if process.returncode == 0:
        return output
    else:
        return f"Error: {error.decode('utf-8')}"

prompt = "Given a sentence, analyze its tone as either 'Positive', 'Negative', or 'Neutral' and summarize the top keywords. Make sure Format your output as follows: 0. [Tone] | 1. [Attitude(verb)] | 2. [Keyword1] | 3. [Keyword2] | 4. [Keyword3] | 5. [Keyword4] | 6. [Keyword5]\n Ensure that the keywords are relevant and capture the essence of the sentence. Try to identify the agents (action initiators) and patients (action recipients) in keywords. Do not explain or give any other content! Just seven tag! You only only only need to give me the tags. The sentence is: "

In [3]:
# show how many lines are in the file
with open('GBcomments.csv') as f:
    lines = f.readlines()
    print(len(lines))

718745


In [21]:
# read the file line by line
# create a dataframe with the new comment and the new comment words
comment_tags_df = pd.DataFrame(columns=['comment', 'tone', 'attitude', 'keyword1', 'keyword2', 'keyword3', 'keyword4', 'keyword5'])
with open('GBcomments.csv') as f:
    lines = f.readlines()
    # skip the first line and read the rest
    # loop through the lines and print the first 5 lines
    for line in lines[50:150]:
        # print the process of the loop every 1000 lines
        if lines.index(line) % 1000 == 0:
            print("Processing line", lines.index(line))
        # skip the content before the first comma, do not use split
        # skip the last two numbers which are the likes and replies
        new_comment = line[line.index(',')+2:line.rindex(',')-3]
        # print the first 5 lines
        query = prompt + new_comment
        response = generate_sentence(query)
        response_data = json.loads(response)
        response_content = response_data["message"]["content"].split('\n')[0].split(' | ')
        new_comment_words = []
        for item in response_content:
            try:
                new_comment_words.append(item.split('. ')[1])
            except IndexError:  
                continue  
        print(new_comment)
        print(new_comment_words)
        # make sure the new comment words are 7 in length, if not, add "None" to the list, if more than 7, ignore the rest
        if len(new_comment_words) < 7:
            new_comment_words += ['None'] * (7 - len(new_comment_words))
        elif len(new_comment_words) > 7:
            new_comment_words = new_comment_words[:7]
        # use concat to add the new comment and the new comment words to the dataframe
        comment_tags_df = pd.concat([comment_tags_df, pd.DataFrame([[new_comment] + new_comment_words], columns=['comment', 'tone', 'attitude', 'keyword1', 'keyword2', 'keyword3', 'keyword4', 'keyword5'])], ignore_index=True)
        
# print the dataframe
print(comment_tags_df.head())
# store the dataframe in a csv file
comment_tags_df.to_csv('GB_comment_tags.csv', index=False)
        

Yo ya me compre el iphone
['Neutral', 'Got', 'iPhone', 'Me', 'Compre', 'Ya', '(None)']
Songs
['Neutral', 'Are', 'Songs', 'None', 'None', 'None', 'None']
Eeee şimdi burda ne olcak , amaç ne
['Negative', 'Worry', 'Burda', 'Olacak', 'Aimak', 'Ne', 'None', 'None']
Good Job steve jobs RIP
['Positive', 'Congratulate', 'Job', 'Steve', 'RIP', 'Good', '(None)', 'None']
Good job. Plz watch realistic artwork. https://youtu.be/6HqJLy97Ras
['Positive', 'Congratulate', 'Job', 'Watch', 'Realistic', 'Artwork', 'Video']
Schlechtes iphone
['Negative', 'Criticize', 'iPhone', 'Bad', 'None', 'None', 'None']
https://youtu.be/Rt1Qx1HssQI
['Neutral', 'Share', 'Video', 'YouTube', 'Link', 'None', 'None']
OMG 25 hours
['Negative', 'Exclaimed', 'Time', 'Hours', 'Long', 'Tired', 'Exhausted']
same as samsang s8
['Positive', 'Praising', 'Samsung', 'S8', 'Same', 'Phone', 'None', 'None']
Fulckoing shoit mo8. Stolp copyinh Samsoung.
['Negative', 'Criticizing', 'Samsung', 'Copying', 'Foul', 'Shoot', 'Stole', 'None']
Eh 

In [10]:
import profanity_check

def predict_profanity(comment):

    new_comment_list = []
    new_comment_list.append(comment)
    if profanity_check.predict(new_comment_list)[0] == 1:
        return 'Profane'
    else:
        return 'Not Profane'

In [6]:
test_comment = "I love this song, it's so good"
print(predict_profanity(test_comment))
test_comment = "I love this song, it's so shit awesome"
print(predict_profanity(test_comment))

Not Profane
Profane


In [12]:
def predict_profanity_llm(comment):
    prompt = "Given a sentence, analyze whether it has bad words (profanity checking) or not. You should only return Profane or Not Profane. Don't explain anything!!! The sentence is: "
    query = prompt + comment
    response = generate_sentence(query)
    response_data = json.loads(response)
    response_content = response_data["message"]["content"]
    return response_content

In [8]:
test_comment = "I love this song, it's so good"
print(predict_profanity_llm(test_comment))
test_comment = "I love this song, it's so fucking awesome"
print(predict_profanity_llm(test_comment))

Not Profane
Profane


In [8]:
profanity_df = pd.read_csv("demo.csv",header=None)

profanity_df = profanity_df.rename(columns={0: 'Tag', 1: 'Content'})
profanity_df.head()

,Tag,Content
0,0,Then go to the village pump and suggest they c...
1,1,Dis hoe wasnt dis violent on Lottery Ticket ðŸ...
2,0,It is better for Atabay not helping the banned...
3,0,"is in CamelCase. ""SiCKO"" is not CamelCase, so..."
4,0,Nonetheless lactose has a hemiacetal group whi...


In [14]:
profanity_df['profanity check'] = profanity_df['Content'].apply(lambda x: predict_profanity(x))
profanity_df['profanity check llm'] = profanity_df['Content'].apply(lambda x: predict_profanity_llm(x))

In [17]:
profanity_df = profanity_df.reindex(columns=['Content', 'Tag', 'profanity check', 'profanity check llm'])
profanity_df.head()
profanity_df.to_csv("demo_result.csv")